In [ ]:
import sys
sys.path.append('../')
sys.path.append('../../')

In [ ]:
import numpy as np
from utils import symbolicMatrixState
from qibo.models import Circuit
from qibo import gates
import qibo
from qibo.models import QFT
qibo.set_backend("tensorflow")

In [ ]:
import tensorflow as tf

## Random circuit - check GPU connection

In [ ]:
circ = QFT(20)
with tf.device("/GPU:0"):
    result = circ.execute(nshots=100)
#print(result.frequencies(binary=True))

## Grover's algorithm

In [ ]:
import scripts.grover as g
import scripts.oracle as o

In [ ]:
def f(x, threshold):
    return x < threshold

In [ ]:
np.binary_repr(2, width=3)

In [ ]:
ket_i0 = np.zeros((2**3, 1))
print(ket_i0.shape)
ket_i0[5] = 1
bra_i0 = np.conj(ket_i0).T

In [ ]:
matrix1 = np.dot(ket_i0, bra_i0)

In [ ]:
matrix2 = np.dot(ket_i0, bra_i0)

In [ ]:
matrix=[67.25294171474508, 341.0046370708946, 361.6244550704071, 54.89410458703692, 37.312050326283725, 286.25991681291254, 44.04787911150442, 106.02284523771786, 35.10230157618279, 80.45737155608685, 71.5131811053048, 103.87453633490934, 43.9400597694365, 57.21587206857912, 218.71116424004526, 32.864633001639476, 89.83296715677876, 43.27725816604674, 32.98375224637114, 36.218838698027085, 311.4535607957833, 66.91939314200651, 41.891298334134014, 288.4773673408346, 56.34191146574942, 58.69523078602758, 70.3560052109315, 35.698261301471845, 164.25841686236774, 41.29339465190191, 292.03426700663215, 35.56992753300877, 110.24105624151994, 51.209652512347446, 34.061933313734365, 39.31883374106132, 137.85947622964898, 40.89703874128519, 285.63854413020715, 246.19816774606602, 113.66295883227889, 197.94149908288352, 53.22957254643204, 56.88739239394056, 263.68492319704444, 42.75375675571238, 145.93154089602046, 279.6278336145787, 177.34298468676246, 50.446407497638376]

In [ ]:
import numpy as np
np.argmin(matrix)

In [ ]:
matrix = np.identity(8)
matrix[0,0] = -1
#matrix[0,1] = -2
#matrix[1,0] = -2
#matrix[1,1,]= -1

In [ ]:
o.is_unitary(imaginary_I)

In [ ]:
imaginary_I = np.matrix(np.identity(2**8, dtype=complex))

In [ ]:
for ix in range(imaginary_I.shape[0]):
    imaginary_I[ix, ix] = 1j
imaginary_I

In [ ]:
nqubits = 7 #checked from 2 to 7

In [ ]:
oracle_circ = o.simple_oracle(nqubits, '101110')

In [ ]:
grover_circ = g.grover_qc(nqubits, oracle_circ)

In [ ]:
counts = grover_circ.execute(nshots=100000).frequencies(binary=True)

In [ ]:
counts

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts)

In [ ]:
# create oracle
n=3
oracle_matrix = np.matrix(np.identity(2**n))
# add phase shift to winner index
i0_idx = int('101', 2)
oracle_matrix[i0_idx, i0_idx] = -1

In [ ]:
m = oracle_matrix
np.allclose(np.identity(m.shape[0]), m.H * m) # check for unitary

In [ ]:
oracle_matrix.shape

## Compare DistCalc circuits

In [ ]:
import numpy as np
import scripts.distance_calc as distc
import h5py

import qibo
qibo.set_backend("tensorflow")

In [ ]:
# dimension 2,3
dim = 2
samples_n = int(200)
data_1 = np.random.multivariate_normal(mean=(0,0), cov=np.eye(dim)*1, size=samples_n)
data_2 = np.random.multivariate_normal(mean=(1,1), cov=np.eye(dim)*1, size=samples_n)

In [ ]:
import importlib
importlib.reload(distc)

In [ ]:
import plots as p
#p.plot_clusters(np.vstack([data_1, data_2]), np.vstack([np.zeros((samples_n,1)), np.ones((samples_n,1))]))
p.plot_latent_representations(np.vstack([data_1, data_2]), np.vstack([np.zeros((samples_n,1)), np.ones((samples_n,1))]))

In [ ]:
dist_1=[]; dist_2=[]
for i in range(samples_n):
    for j in range(samples_n):
        arr1 = []
        
        distance, _ = distc.DistCalc_AmplE(data_1[i,:], data_2[j,:])
        arr1.append(distance)
    
        arr2 = []
        distance, _ = distc.DistCalc_DI(data_1[i,:], data_2[j,:])
        arr2.append(distance)
    dist_1.append(np.sum(arr1))
    dist_2.append(np.sum(arr2))

In [ ]:
dist=[]
for i in range(samples_n):
    for j in range(samples_n):
        arr = []
        
        distance = np.linalg.norm(data_1[i,:] - data_2[j,:])
        arr.append(distance)
    dist.append(np.sum(arr))

In [ ]:
dist = np.array(dist)
dist_1 = np.array(dist_1)
dist_2 = np.array(dist_2)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.hist(np.abs(dist_1-dist_2), histtype = 'step', fill=False, linewidth=2, bins=100, label='AmplE - DI(new)', density=True, color='forestgreen')
plt.hist(np.abs(dist-dist_2), histtype = 'step', fill=False, linewidth=2, bins=50, label='DI(new) - Euclid', density=True, color='deeppink')
plt.hist(np.abs(dist_1-dist), histtype = 'step', fill=False, linewidth=2, bins=100, label='AmplE - Euclid', density=True, color='lightblue')

plt.yscale('log')
plt.legend(frameon=True, prop={"size":15})
plt.title(f'Abs Difference')
plt.show()

In [ ]:
import scipy

plt.figure(figsize=(10,6))
plt.hist(scipy.stats.zscore(dist_1-dist_2), histtype = 'step', fill=False, linewidth=2, bins=50, label='AmplE - DI(new)', density=True, color='forestgreen')
plt.hist(scipy.stats.zscore(dist_2-dist), histtype = 'step', fill=False, linewidth=2, bins=50, label='DI(new) - Euclid', density=True, color='deeppink')
plt.hist(scipy.stats.zscore(dist_1-dist), histtype = 'step', fill=False, linewidth=2, bins=50, label='AmplE - Euclid', density=True, color='lightblue')

plt.yscale('log')
plt.legend(frameon=True, prop={"size":15})
plt.title(f'Z-score')
plt.show()

## Check DI circuit with Qiskit

In [ ]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, Aer, IBMQ, execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.visualization import plot_bloch_multivector, plot_histogram

from scripts.util import create_gate, prepare_input, normalize, calc_norm
import numpy as np
from scripts.distance_calc import pad_input

In [ ]:
a = np.array([1,2,3,4]); b=np.array([2,3,4,5])

In [ ]:

num_features = len(a)
norm = calc_norm(a, b)
a_norm = a/norm
b_norm = b/norm

a_norm = pad_input(a_norm)
b_norm = pad_input(b_norm)

amplitudes = np.concatenate((a_norm, b_norm))
n_qubits = int(np.log2(len(amplitudes)))

qr_phi = QuantumRegister(n_qubits, "phi") # size always 1
cr = ClassicalRegister(1, "cr")

qc_phi = QuantumCircuit(qr_phi, cr)
qc_phi.initialize(amplitudes)
qc_phi.h(0)

qc_phi.measure(0, cr)

qc_phi.draw()

In [ ]:
# To use local qasm simulator
import math
backend = Aer.get_backend('qasm_simulator')
shots_n = 10000
counts = execute(qc_phi, backend=backend, shots=shots_n).result().get_counts(qc_phi)

distance = norm*math.sqrt(2)*math.sqrt((counts['1']/shots_n))

In [ ]:
distance

In [ ]:
from scripts.distance_calc import DistCalc_DI

distance_di, _ = DistCalc_DI(a, b)
distance_di